# ASSIGNMENT 1 SOLUTION

## Part 1

### k-Nearest Neighbor Classification

Answer 1 :

kNN wouldn't be a great idea in larger datasets because;

1: It is computationally expensive since in a large dataset the algorithm of the kNN would be time-consuming and memory-intensive.

2: kNN is sensitive to irrelevant data, outliers and high dimensions. These matters would make it hard for kNN to produce good results, making the model impractical.

Answer 2:

The optimal value of k is 10 because it has the lowest validation error.
It is overfitting for the values lower than 10 and underfitting for the values higher.

Answer 3:

K = 1 wouldn't be suitable because it would overfit the data since it predicts the 1st nearest neighbor for each prediction.

K = 3 would be better because model will be more generalized since it takes 3 neighbors in calculation.

K = 5 wouldn't be suitable because model will underfit the data, model would generalize too much.

Answer 4:

If all instances of the data have the same scale then k-Nearest Neighbor’s performance increases drastically. (T)

While k-Nearest Neighbor performs well with a small number of input variables, it’s performance decreases when the number of inputs becomes large. (T)

k-Nearest Neighbor makes supposes nothing about the functional form of the
problem it handles. (T)

### Linear Regression

Answer 1:

In [ ]:
x2_real = 2025
sum_x2 = 7569 + 4900 + 8464 + 4489 + 2025
x2_normalized = x2_real / sum_x2
print("Normalized value of x2(5) is %f"%x2_normalized)

Normalized value of x2(5) is 0.073779


Answer 2:

Perpendicular offset is better because we use the shortest distance.

Answer 3:

In [ ]:
import numpy as np

# Define the feature matrix X and the target vector y
x = np.array([[1, 1], [1, 2], [1, 4], [1, 0]])
y = np.array([0.5, 1, 2, 0])

x_transpose = x.transpose()  # [[1, 1, 1, 1], [1, 2, 4, 0]]

# Calculate X^T * X
x_transpose_multiply_x = np.dot(x_transpose, x)

# Calculate the inverse of X^T * X
invert_x_t_x = np.linalg.inv(x_transpose_multiply_x)

# Calculate X^T * y
x_transpose_multiply_y = np.dot(x_transpose, y)

# Calculate the model coefficients θ
model = np.dot(invert_x_t_x, x_transpose_multiply_y)
print("θ1 and θ2 are :",end = " ")
print(model)

θ1 and θ2 are : [2.77555756e-16 5.00000000e-01]


Answer 4:

Using feature scaling, features will contribute equally to dataset thanks to weights and normalization. If different features have different scalings;using a distance-based model like kNN would make features that have larger numerical values affect more the model trained. Scaling also leads to faster convergence of the model.

## Part 2

## Anime Recommendation System

Import necessary libraries and authenticate the user for the drive link provided

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
import pandas as pd
import zipfile
import io

auth.authenticate_user()
google_auth = GoogleAuth()
google_auth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(google_auth)


Define necessary functions to read content directly from the link without downloading it

In [ ]:
def read_content_from_drive(file_id,file_name):
  downloaded = drive.CreateFile({'id' : file_id})
  downloaded.GetContentFile(file_name)

In [ ]:
def read_content_from_zipfile(file):
  with zipfile.ZipFile("Dataset_A1.zip","r") as zip_file:
    with zip_file.open(file) as file_raw:
      file_data = file_raw.read()
    df = pd.read_csv(io.BytesIO(file_data))
  return df

Read the contents by calling the functions

In [ ]:
read_content_from_drive("1kCL8dZLHQUlBUzH-DmYHRNi7xMLrgS4b","Dataset_A1.zip")
animes = read_content_from_zipfile("animes.csv")
user_rates_train = read_content_from_zipfile("user_rates_train.csv")
user_rates_test = read_content_from_zipfile("user_rates_test.csv")

### Preprocessing

We do all the process for both datasets

Extract Anime names, Usernames,Studios and Image URL from data

In [ ]:
animes.drop(["Image URL","Name","Studios"],axis = 1,inplace = True)
user_rates_train.drop(["Username","Anime Title"], axis = 1, inplace = True)
user_rates_test.drop(["Username","Anime Title"], axis = 1, inplace = True)

#### All process is done for the both test and train datasets

Merge the user rates and animes dataset under 'anime_id' column

In [ ]:
df_train = pd.merge(animes,user_rates_train, on = "anime_id", how = "inner")
df_test = pd.merge(animes,user_rates_test, on = "anime_id", how = "inner")

Adjust the Genre column to encode

In [ ]:
#For df_test
genre_split = df_test['Genres'].str.split(', ', expand=True) #Split the line into 3 elements
genre_split = genre_split.iloc[:,:3]
column_names = {0 : 'Genre1',1 : 'Genre2', 2 : 'Genre3'}  #Arrange the dataframe for Genres
genre_split = genre_split.rename(columns = column_names)
df_test = pd.concat([df_test,genre_split],axis = 1) #Concatenate the dataframes
df_test = df_test.drop(columns = ['Genres'])


#For df_train
genre_split = df_train['Genres'].str.split(', ', expand=True)
genre_split = genre_split.iloc[:,:3]
column_names = {0 : 'Genre1',1 : 'Genre2', 2 : 'Genre3'}
genre_split = genre_split.rename(columns = column_names)
df_train = pd.concat([df_train,genre_split],axis = 1)
df_train = df_train.drop(columns = ['Genres'])

Adjust the Duration column to encode

In [ ]:
#For df_train
duration_list = []   #Arrange the format of the duration cells
for i in df_train['Duration']:
  cell_list = i.split(" ")
  if cell_list[1] == 'hr':
    duration_list.append(int(cell_list[0]) * 60)
  elif cell_list[1] == 'min':
    duration_list.append(int(cell_list[0]))
  else:
    duration_list.append(0.5)
df_train = df_train.drop(columns = ['Duration'])
duration_df = pd.DataFrame(duration_list,columns = ['Duration'])
df_train = pd.concat([df_train,duration_df] , axis = 1)



#For df_test
duration_list = []
for i in df_test['Duration']:
  cell_list = i.split(" ")
  if cell_list[1] == 'hr':
    duration_list.append(int(cell_list[0]) * 60)
  elif cell_list[1] == 'min':
    duration_list.append(int(cell_list[0]))
  else:
    duration_list.append(0.5)
df_test = df_test.drop(columns = ['Duration'])
duration_df = pd.DataFrame(duration_list,columns = ['Duration'])
df_test = pd.concat([df_test,duration_df] , axis = 1)

Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

columns_to_encode = ['Type','Source',"Genre1","Genre2","Genre3"]
for col in columns_to_encode:
  df_train[col] = label_encoder.fit_transform(df_train[col])


for col in columns_to_encode:
  df_test[col] = label_encoder.fit_transform(df_test[col])



### Split both datasets


In [ ]:
x_train = df_train.drop('rating',axis = 1)
y_train = df_train['rating']

x_test = df_test.drop('rating',axis = 1)
y_test = df_test['rating']

Define a function that implements kNN and calculates Mean Absolute Error

In [ ]:
def knn_and_error(k):

  total_error = 0

  for test_index in range(len(x_test)):

    vector1 = x_test.iloc[test_index]

    similarities = {}

    for train_index in range(len(x_train)):
      vector2 = x_train.iloc[train_index]
      similarities[train_index] = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)) #Calculate the similarities and add them to dictionaries.

    similarities = dict(sorted(similarities.items(), key=lambda item: item[1],reverse = True))

    top_k_similarities = (list(similarities.items())[:k])   #Get top k similarities

    predicted_rate = 0      #Calculate the errors

    for similarity_score in top_k_similarities:
      predicted_rate += y_train.iloc[similarity_score[0]]

    error = abs((predicted_rate/k) - y_test.iloc[test_index])
    total_error += error

  mean_absolute_error = total_error / len(x_test)
  return f"Mean absolute error for k = {k} is {mean_absolute_error}"

Report the results for k = 3, 5, 7

In [ ]:
print(knn_and_error(3))

Mean absolute error for k = 3 is 1.359179019384265


In [ ]:
print(knn_and_error(5))

Mean absolute error for k = 5 is 1.2884834663626017


In [ ]:
print(knn_and_error(7))

Mean absolute error for k = 7 is 1.2480860074930784


## Report

### Introduction

In this report, we present the development and evaluation of an anime recommendation system using a K-Nearest Neighbors (KNN) algorithm based on cosine similarity. The goal of this project was to recommend anime titles to users based on their viewing history and user ratings. The dataset consisted of two parts: an anime dataset with anime information and a user_rating dataset with user ratings. We divided the data into training and testing sets to evaluate the recommendation algorithm's performance.


### Data Preprocessing

We loaded the anime dataset and user_rating dataset separately.

Split the data into training and testing sets.

Handled missing values and data cleaning.

Ensured that the user_rating dataset was appropriately aligned with the anime dataset.

### K-Nearest Neighbors Algorithm:

We implemented a KNN algorithm using cosine similarity.

The K parameter (number of nearest neighbors) was a crucial hyperparameter to determine the performance of the algorithm.

### Model Evaluation:

We evaluated the KNN recommendation algorithm on the test dataset.

We varied the value of K to identify the optimal K for the lowest Mean Absolute Error (MAE).


### Results


After experimenting with different values of K, we observed that K = 7 produced the lowest Mean Absolute Error (MAE) when comparing the predicted user ratings with the actual user ratings. This suggests that considering the seven nearest neighbors produced the most accurate anime recommendations based on user preferences and viewing history.

#### Conclusion

In this project, we successfully developed an anime recommendation system using the K-Nearest Neighbors algorithm with cosine similarity. We concluded that setting K to 7 provides the best recommendation performance, as it resulted in the lowest MAE on our test dataset. This means that considering the seven nearest neighbors when making recommendations led to more accurate and personalized anime recommendations for users. The system can be further improved and deployed for anime enthusiasts to enhance their viewing experiences. Further refinements could include incorporating user-based or item-based collaborative filtering, or even leveraging more advanced recommendation algorithms.
